In [30]:
import requests
import datetime
import json
import csv
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import traceback
import logging
import seaborn as sns
from pandas.io.json import json_normalize
%matplotlib inline

import sys
default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.stdout = default_stdout
sys.stderr = default_stderr
sys.setdefaultencoding('utf-8')



In [98]:
def check_connect_output_json(url_address):
    limit_retry_number = 20
    for i in range(limit_retry_number):
        try :
            r = requests.get(url_address)
            if r.status_code == 200:
                r.encoding = "utf-8"
                jsondata = r.json()
                break
            else:
                print "retry ",i," times! 5sec between and retry only 20 times."
                print "else sleep"
                time.sleep(5)
        except (KeyboardInterrupt, SystemExit):
            raise
        except Exception as e:
            print "retry ",i," times! 5sec between and retry only 20 times."
            print "except sleep"
            time.sleep(5)
        if i == 20:
            return {"this url":{"is bad"}}
    return jsondata

def get_page_posts_to_df(url,page_num):
    
    jsondata = check_connect_output_json(url)['posts']
    testdf = pd.DataFrame()
    # 只要有分頁連結就一直往下
    x=0
    while 'paging' in jsondata:
        x +=1
        temp=json_normalize(jsondata['data'])
        temp = temp.drop(temp.ix[:,[0,7]],axis =1)
        id_list=temp["id"].tolist()
        
        append_temp = pd.DataFrame()
        for post_id in id_list:
            react_url=reactions_number(post_id,access_token)
            react_json=check_connect_output_json(react_url)
            react_temp=json_normalize(react_json)
            if len(react_temp.columns)== 13:
                react_temp = react_temp.drop(react_temp.ix[:,[0,2,5,7,9,11]],axis =1)
                react_temp.columns=['angry','haha','id','like','love','sad','wow']
            else:
                react_temp.assign(angry=0,haha=0,id=temp['id'],like=0,love=0,sad=0,wow=0)
          
            append_temp=append_temp.append(react_temp,ignore_index=True)
        
        merge_temp = pd.merge(append_temp,temp,on= "id")
        
        testdf=testdf.append(merge_temp,ignore_index=True)
        newurl = jsondata['paging']['next']
        newr= requests.get(newurl)
        newr.encoding="utf-8"
        jsondata = newr.json()
        print x,time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

        #若超過幾個分頁就先暫停
        if x==page_num:
            break
#     cl=['angry','haha','id','like','love','sad','wow','comments','create_time','link','message','name','reactions','shares','type']
#     testdf.columns= cl
    # 先不改時間格式，等全部抓下來再一起改，免得出錯。
#     testdf.create_time =pd.to_datetime(testdf.create_time,format= "%Y-%m-%dT%H:%M:%S+0000")
    
#     od=['id','type','create_time','link','message','name','angry','haha','like','love','sad','wow','reactions','comments','shares']
#     testdf=testdf[od]
    print "done"
    return testdf

In [99]:
def reactions_number(status_id,access_token):
    status_id = status_id

    base = "https://graph.facebook.com/v2.8"
    node = "/%s" % status_id
    reactions = "/?fields=" \
            "reactions.type(LIKE).limit(0).summary(total_count).as(like)" \
            ",reactions.type(LOVE).limit(0).summary(total_count).as(love)" \
            ",reactions.type(WOW).limit(0).summary(total_count).as(wow)" \
            ",reactions.type(HAHA).limit(0).summary(total_count).as(haha)" \
            ",reactions.type(SAD).limit(0).summary(total_count).as(sad)" \
            ",reactions.type(ANGRY).limit(0).summary(total_count).as(angry)"
    parameters = "&"+access_token
    url = base + node + reactions + parameters
#     print url
    return url
def fb_pages_info(page_id,access_token):
    # 選API版本預設2.8
    fb_graph_api = "https://graph.facebook.com/v2.8/"
    # 放粉絲頁ID
    nodes = page_id
    # access_token
    access_token = access_token
    # 所要看的欄位

    fields ="""/?fields=posts{id,name,link,message,shares,created_time,type,comments.limit(0).summary(total_count),reactions.limit(0).summary(total_count)}""" 
    # 把url連接整理好丟出去
    url = fb_graph_api+nodes+fields+"&"+access_token
    # status_id_url_end = fields+"&"+access_token
#     print url
    return url

app_id = "393833577650615"
app_secret = "3ab280dac70995e4d8e601cbc025b53a"

access_token = "access_token="+app_id + "|" + app_secret

page_id_list = ['YaoTurningTaipei','DoctorKoWJ','appledaily.tw',"gamer.com.tw","crazyck101","WoTBlitzTW"]

page_id = "WoTBlitzTW"

url = fb_pages_info(page_id,access_token)



In [102]:
r = requests.get(url)

r.encoding = "utf-8"
jsondata = r.json()['posts']
temp=json_normalize(jsondata['data'])
temp

comments.data  comments.summary.total_count              created_time  \
0             []                             6  2017-04-14T11:02:20+0000   
1             []                             1  2017-04-11T10:30:00+0000   
2             []                             1  2017-04-11T05:48:35+0000   
3             []                             3  2017-04-10T11:34:33+0000   
4             []                             0  2017-04-10T10:00:00+0000   
5             []                             7  2017-04-09T02:57:24+0000   
6             []                             0  2017-04-08T08:02:04+0000   
7             []                             0  2017-04-07T06:53:00+0000   
8             []                             0  2017-04-06T13:09:29+0000   
9             []                             8  2017-04-06T13:00:00+0000   
10            []                            12  2017-04-06T10:00:00+0000   
11            []                             0  2017-04-06T08:00:54+0000   
12            []                             0  2017-04-05T10:30:32+0000   
13            []                             0  2017-04-04T12:04:55+0000   
14            []                             0  2017-04-04T10:30:31+0000   
15            []                             0  2017-04-03T13:29:06+0000   
16            []                            10  2017-04-03T12:00:00+0000   
17            []                             0  2017-04-03T10:30:31+0000   
18            []                             2  2017-04-02T13:30:30+0000   
19            []                             2  2017-04-02T12:00:00+0000   
20            []                            10  2017-04-01T12:00:00+0000   
21            []                             5  2017-04-01T10:30:00+0000   
22            []                             0  2017-03-31T13:06:55+0000   
23            []                             8  2017-03-31T10:24:13+0000   
24            []                             6  2017-03-31T09:46:33+0000   

                                   id  \
0   1504885679727769_1319557104764576   
1   1504885679727769_1966286333587699   
2   1504885679727769_1969919523224380   
3   1504885679727769_1969498749933124   
4   1504885679727769_1969439026605763   
5   1504885679727769_1968854859997513   
6   1504885679727769_1968287510054248   
7   1504885679727769_1967615073454825   
8   1504885679727769_1967109416838724   
9   1504885679727769_1967017963514536   
10  1504885679727769_1966970706852595   
11  1504885679727769_1966975906852075   
12  1504885679727769_1966325940250405   
13  1504885679727769_1965804080302591   
14  1504885679727769_1965767023639630   
15  1504885679727769_1965285867021079   
16  1504885679727769_1962624660620533   
17  1504885679727769_1965213573694975   
18  1504885679727769_1964726420410357   
19  1504885679727769_1962626687286997   
20  1504885679727769_1963203200562679   
21  1504885679727769_1963775127172153   
22  1504885679727769_1963344887215177   
23  1504885679727769_1963284237221242   
24  1504885679727769_1963266467223019   

                                                 link  \
0   https://www.facebook.com/WoTBlitzTW/videos/131...   
1   https://www.facebook.com/WoTBlitzTW/photos/a.1...   
2   https://www.facebook.com/WoTBlitzTW/photos/a.1...   
3   https://www.facebook.com/WoTBlitzTW/photos/a.1...   
4   https://www.facebook.com/WoTBlitzTW/photos/a.1...   
5   https://www.facebook.com/WoTBlitzTW/photos/a.1...   
6   https://www.facebook.com/WoTBlitzTW/photos/a.1...   
7   https://www.facebook.com/WoTBlitzTW/photos/a.1...   
8   https://www.facebook.com/WoTBlitzTW/photos/a.1...   
9   https://www.facebook.com/WoTBlitzTW/photos/a.1...   
10  https://www.facebook.com/WoTBlitzTW/photos/a.1...   
11  https://www.facebook.com/WoTBlitzTW/photos/a.1...   
12  https://www.facebook.com/WoTBlitzTW/photos/a.1...   
13   https://www.facebook.com/events/478182219239666/   
14  https://www.facebook.com/WoTBlitzTW/photos/a.1...   
15  https://www.facebook.com/WoTBlitzTW/videos/

In [100]:
# testdf = pd.DataFrame()
datadf = get_page_posts_to_df(url,100)
# datadf.to_csv(page_id+'.csv',index=False)
# datadf.plot(datadf.create_time,figsize=(12,8))

1 2017-04-16 22:57:27
2 2017-04-16 22:57:35
3 2017-04-16 22:57:42
4 2017-04-16 22:57:50
5 2017-04-16 22:57:58
6 2017-04-16 22:58:06
7 2017-04-16 22:58:13
8 2017-04-16 22:58:21
9 2017-04-16 22:58:28
10 2017-04-16 22:58:36
11 2017-04-16 22:58:43
12 2017-04-16 22:58:51
13 2017-04-16 22:58:59
14 2017-04-16 22:59:06
15 2017-04-16 22:59:14
16 2017-04-16 22:59:21
17 2017-04-16 22:59:29
18 2017-04-16 22:59:36
19 2017-04-16 22:59:44
20 2017-04-16 22:59:52
21 2017-04-16 22:59:59
22 2017-04-16 23:00:07
23 2017-04-16 23:00:14
24 2017-04-16 23:00:22
25 2017-04-16 23:00:29
26 2017-04-16 23:00:37
27 2017-04-16 23:00:44
28 2017-04-16 23:00:52
29 2017-04-16 23:00:59
30 2017-04-16 23:01:07
31 2017-04-16 23:01:14
32 2017-04-16 23:01:22
33 2017-04-16 23:01:29
34 2017-04-16 23:01:37
35 2017-04-16 23:01:44
36 2017-04-16 23:01:52
37 2017-04-16 23:01:59
38 2017-04-16 23:02:00
done


In [105]:

datadf = datadf.drop(datadf.ix[:,[10]],axis =1)

In [108]:
datadf

angry  comments              created_time  haha  \
0        0         6  2017-04-14T11:02:20+0000     0   
1        0         1  2017-04-11T10:30:00+0000     0   
2        0         1  2017-04-11T05:48:35+0000     0   
3        0         3  2017-04-10T11:34:33+0000     0   
4        0         0  2017-04-10T10:00:00+0000     0   
5        0         7  2017-04-09T02:57:24+0000     0   
6        0         0  2017-04-08T08:02:04+0000     0   
7        0         0  2017-04-07T06:53:00+0000     0   
8        0         0  2017-04-06T13:09:29+0000     0   
9        1         8  2017-04-06T13:00:00+0000     1   
10       0        12  2017-04-06T10:00:00+0000     3   
11       0         0  2017-04-06T08:00:54+0000     1   
12       0         0  2017-04-05T10:30:32+0000     0   
13       0         0  2017-04-04T12:04:55+0000     0   
14       0         0  2017-04-04T10:30:31+0000     0   
15       0         0  2017-04-03T13:29:06+0000     0   
16       0        10  2017-04-03T12:00:00+0000     1   
17       0         0  2017-04-03T10:30:31+0000     0   
18       0         2  2017-04-02T13:30:30+0000     0   
19       0         2  2017-04-02T12:00:00+0000     0   
20       9        10  2017-04-01T12:00:00+0000    13   
21       0         5  2017-04-01T10:30:00+0000     0   
22       0         0  2017-03-31T13:06:55+0000     0   
23       0         8  2017-03-31T10:24:13+0000    32   
24       3         6  2017-03-31T09:46:33+0000     0   
25       0         0  2017-03-30T14:22:59+0000     0   
26       0         0  2017-03-30T12:26:25+0000     0   
27       0        14  2017-03-30T11:30:14+0000     2   
28       0         9  2017-03-29T12:00:17+0000     1   
29       0         3  2017-03-25T10:27:11+0000     0   
..     ...       ...                       ...   ...   
898      0         4  2014-08-07T03:54:03+0000     0   
899      0        21  2014-08-05T07:00:01+0000     0   
900      0         3  2014-08-04T09:00:00+0000     0   
901      0         0  2014-08-04T07:02:12+0000     0   
902      0         6  2014-08-03T11:27:54+0000     0   
903      0        11  2014-08-01T14:00:00+0000     0   
904      0         3  2014-08-01T12:07:29+0000     0   
905      0        10  2014-08-01T04:00:00+0000     0   
906      0         4  2014-07-30T09:20:00+0000     0   
907      0         0  2014-07-28T07:02:06+0000     0   
908      0         5  2014-07-28T03:07:19+0000     0   
909      0         0  2014-07-25T09:00:00+0000     0   
910      0         4  2014-07-25T05:30:00+0000     0   
911      0         7  2014-07-25T04:00:40+0000     0   
912      0         1  2014-07-24T11:47:10+0000     0   
913      0        13  2014-07-23T08:32:18+0000     0   
914      0         4  2014-07-23T04:19:01+0000     0   
915      0         7  2014-07-22T10:46:56+0000     0   
916      0        13  2014-07-22T06:34:24+0000     0   
917      0         2  2014-07-21T08:05:33+0000     0   
918      0         4  2014-07-16T12:00:01+0000     0   
919      0         3  2014-07-16T07:59:57+0000     0   
920      0         3  2014-07-11T06:48:21+0000     0   
921      0         3  2014-07-09T04:46:34+0000     0   
922      0         0  2014-07-07T10:39:02+0000     0   
923      0         0  2014-07-07T07:00:27+0000     0   
924      0         0  2014-07-03T09:07:27+0000     0   
925      0         1  2014-07-02T05:27:35+0000     0   
926      0         0  2014-06-26T08:51:00+0000     0   
927      0         0  2014-06-26T07:00:00+0000     0   

                                    id  like  \
0    1504885679727769_1319557104764576    74   
1    1504885679727769_1966286333587699    51   
2    1504885679727769_1969919523224380    52   
3    1504885679727769_1969498749933124    18   
4    1504885679727769_1969439026605763    35   
5    1504885679727769_1968854859997513    58   
6    1504885679727769_1968287510054248    40   
7    1504885679727769_1967615073454825    21   
8    1504885679727769_1967109416838724    12   
9    1504885679727769_1967017963514536   103   
10  

In [107]:
datadf.columns=['angry','comments','created_time','haha','id','like','link','love','message','name','reactions','sad','shares','type','wow']

In [ ]:
datadf=datadf[['angry','comments','created_time','haha','id','like','link','love','message','name','reactions','sad','shares','type','wow']]

In [75]:
fb_graph_api = "https://graph.facebook.com/v2.8"

page_id = "crazyck101"
edges = 'posts'
fields = "/?fields=message,link,created_time,type,name,id," + \
            "comments.limit(0).summary(true),shares,reactions" + \
            ".limit(0).summary(true)&limit=100&"
nodes = "/"+page_id+"/"+edges
app_id = "393833577650615"
app_secret = "3ab280dac70995e4d8e601cbc025b53a"
access_token = app_id + "|" + app_secret

url = fb_graph_api+nodes+fields+"access_token=%s"%(access_token)
print url


https://graph.facebook.com/v2.8/crazyck101/posts/?fields=message,link,created_time,type,name,id,comments.limit(0).summary(true),shares,reactions.limit(0).summary(true)&limit=100&access_token=393833577650615|3ab280dac70995e4d8e601cbc025b53a


In [41]:
status_id='1021581417961885_1194952983958060'

base = "https://graph.facebook.com/v2.8"
node = "/%s" % status_id
reactions = "/?fields=" \
        "reactions.type(LIKE).limit(0).summary(total_count).as(like)" \
        ",reactions.type(LOVE).limit(0).summary(total_count).as(love)" \
        ",reactions.type(WOW).limit(0).summary(total_count).as(wow)" \
        ",reactions.type(HAHA).limit(0).summary(total_count).as(haha)" \
        ",reactions.type(SAD).limit(0).summary(total_count).as(sad)" \
        ",reactions.type(ANGRY).limit(0).summary(total_count).as(angry)"
parameters = "&access_token=%s" % access_token
url = base + node + reactions + parameters